# FL Client over Secure RPC

In this notebook, we will present how to launch a gRPC client as an FL client with an authenticator. To pair with the server notebook, we consider only one client.

In [1]:
num_clients = 1

## Load client configurations

We load the configuration for the client from `examples/resources/configs/mnist/client_1.yaml`

In [2]:
from omegaconf import OmegaConf

client_config_file = "../../examples/resources/configs/mnist/client_1.yaml"
client_config = OmegaConf.load(client_config_file)
print(OmegaConf.to_yaml(client_config))

client_id: Client1
train_configs:
  device: cpu
  logging_output_dirname: ./output
  logging_output_filename: result
data_configs:
  dataset_path: ./resources/dataset/mnist_dataset.py
  dataset_name: get_mnist
  dataset_kwargs:
    num_clients: 2
    client_id: 0
    partition_strategy: class_noniid
    visualization: true
    output_dirname: ./output
    output_filename: visualization.pdf
comm_configs:
  grpc_configs:
    server_uri: localhost:50051
    max_message_size: 1048576
    use_ssl: false



💡 We need to change the relative path in `data_configs.dataset_path` to point to the right file relative to this notebook.

💡 We also need to change `data_configs.dataset_kwargs.num_clients` to 1 to make sure we only partition the MNIST dataset to one client split. We change `data_configs.dataset_kwargs.visualization` to False as well.

In [3]:
client_config.data_configs.dataset_path = (
    "../../examples/resources/dataset/mnist_dataset.py"
)
client_config.data_configs.dataset_kwargs.num_clients = num_clients
client_config.data_configs.dataset_kwargs.visualization = False

## Create secure SSL channel and authenticator

The client requires a root certificate to verify the server certificate. In this example, we provide that [root certificate](https://github.com/APPFL/APPFL/blob/main/src/appfl/comm/grpc/credentials/root.crt), assuming that the server uses self-signed [certificate](https://github.com/APPFL/APPFL/blob/main/src/appfl/comm/grpc/credentials/localhost.crt) and [key](https://github.com/APPFL/APPFL/blob/main/src/appfl/comm/grpc/credentials/localhost.key) provided by gRPC official documentation.

💡 Please check this [tutorial](https://appfl.ai/en/latest/tutorials/examples_ssl.html) for more details on how to generate SSL certificates for securing the gRPC connections in practice.

To use the provided root certificate, user just to need to set the following. If the user would like to use his own root certificate, just change this to the file path to the local root certificate.

In [4]:
client_config.comm_configs.grpc_configs.use_ssl = True
client_config.comm_configs.grpc_configs.root_certificate = (
    "../../src/appfl/comm/grpc/credentials/root.crt"
)

We also need to set configurations to use the naive authenticator and provide the `auth_token` agreed with the server for authentication.

In [5]:
client_config.comm_configs.grpc_configs.use_authenticator = True
client_config.comm_configs.grpc_configs.authenticator = "NaiveAuthenticator"
client_config.comm_configs.grpc_configs.authenticator_args = {
    "auth_token": "A_SECRET_DEMO_TOKEN"
}

## Create the client agent and communicator

Now we are ready to create the client agent using the `client_agent` defined and modified above, as well as a `GRPCClientCommunicator` to send request to the server.

⚠️ Please make sure that you have started the server from the other notebook!

In [6]:
from appfl.agent import ClientAgent
from appfl.comm.grpc import GRPCClientCommunicator

client_agent = ClientAgent(client_agent_config=client_config)
client_communicator = GRPCClientCommunicator(
    client_id=client_agent.get_id(),
    **client_config.comm_configs.grpc_configs,
)

[2024-12-25 21:13:12,868 INFO Client1]: Logging to ./output/result_Client1_2024-12-25-21-13-12.txt


## Start the FL experiment

Client start the FL experiment by doing the following things:

- Obtain general client-side configurations from the server and load them
- Obtain the initial global model from the server
- *[Optional]* Send the number of local data to the server
- Iteratively train the model and update the global model until receiving a `DONE` status flag from the server.

💡 The server is also logging several information regarding the recipe of client requests.

In [7]:
# Obtain general client-side configurations from the server and load them
client_config = client_communicator.get_configuration()
client_agent.load_config(client_config)

# Obtain the initial global model from the server
init_global_model = client_communicator.get_global_model(init_model=True)
client_agent.load_parameters(init_global_model)

# Send the number of local data to the server
sample_size = client_agent.get_sample_size()
client_communicator.invoke_custom_action(
    action="set_sample_size", sample_size=sample_size
)

while True:
    client_agent.train()
    local_model = client_agent.get_parameters()
    if isinstance(local_model, tuple):
        local_model, meta_data_local = local_model[0], local_model[1]
    else:
        meta_data_local = {}
    new_global_model, metadata = client_communicator.update_global_model(
        local_model, **meta_data_local
    )
    if metadata["status"] == "DONE":
        break
    client_agent.load_parameters(new_global_model)
client_communicator.invoke_custom_action(action="close_connection")

[2024-12-25 21:13:19,453 INFO Client1]:      Round   Pre Val?       Time Train Loss Train Accuracy   Val Loss Val Accuracy
[2024-12-25 21:13:20,469 INFO Client1]:          0          Y                                          2.3006      15.9300
[2024-12-25 21:13:23,361 INFO Client1]:          0          N     2.8921     0.0661        81.3125     0.1673      95.1700
[2024-12-25 21:13:24,419 INFO Client1]:          1          Y                                          0.1673      95.1700
[2024-12-25 21:13:27,223 INFO Client1]:          1          N     2.8028     0.0172        95.0938     0.1102      96.5900
[2024-12-25 21:13:28,299 INFO Client1]:          2          Y                                          0.1102      96.5900
[2024-12-25 21:13:31,202 INFO Client1]:          2          N     2.9023     0.0122        96.3438     0.0753      97.7500
[2024-12-25 21:13:32,264 INFO Client1]:          3          Y                                          0.0753      97.7500
[2024-12-25 21:1

{}